In [1]:
import pandas as pd
import pybasilica.run as run
import torch
import pyro
import numpy as np
import seaborn as sns
import sklearn.metrics

%load_ext autoreload
%autoreload 2

/Users/elenab/Library/r-miniconda-arm64/envs/basilica-env/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [91]:
def mutual_info(x,y):
    assert len(x) == len(y)
    N = len(x)

    # Obtain all possible classes
    x_classes = list(np.unique(x))
    y_classes = list(np.unique(y))
    classes = set(x_classes + y_classes)

    # Compute cardinality of every classes
    px, py, pxy = {}, {}, {}
    for c in classes:
        px[c] = np.count_nonzero(x == c)
        py[c] = np.count_nonzero(y == c)

        pxy[c] = len(set(np.argwhere(x==c)[:,0]).intersection(set(np.argwhere(y==c)[:,0])))

    print(px, py)

    I = 0
    for cx in x_classes:
        for cy in y_classes:
            pxy = len(set(np.argwhere(x==cx)[:,0]).intersection(set(np.argwhere(y==cy)[:,0])))
            print(pxy)

            if pxy > 0:
                print(pxy / N)
                print(np.log(N * pxy / (px[cx] * py[cy])))
                print(N * pxy)
                I += pxy / N * np.log(N * pxy / (px[cx] * py[cy]))

    return I

In [96]:
a = [0 for _ in range(10000)]
b = [0 for _ in range(9999)] + [1 for _ in range(1)] 
sklearn.metrics.normalized_mutual_info_score(a, b) 

0.0

In [11]:
m_path = "test_simul_counts.csv"
m_g = pd.read_csv(m_path)
m = m_g.drop(["groups", "groups_rare"], axis=1)
g = m_g["groups"].tolist()
g_rare = m_g["groups_rare"].tolist()
ref_path = "COSMIC_filt.csv"
cosmic = pd.read_csv(ref_path, index_col=0)
k_list = [4,5]  # true is 4
cluster = [4]  # true is 3 + 1 rare 

In [ ]:
sns.displot(pyro.distributions.HalfNormal(0.5).sample((100,)).tolist(), bins=100, binrange=(0,15))
sns.displot(pyro.distributions.HalfCauchy(0.5).sample((100,)).tolist(), bins=100, binrange=(0,15)) 

In [ ]:
# sns.displot(pyro.distributions.Normal(0.3, 0.05).sample((100,)).tolist(), bins=100, binrange=(0,20)) 
# sns.displot(pyro.distributions.StudentT(0.3, 0.05).sample((100,)).tolist(), bins=100, binrange=(0,20)) 
# sns.displot(pyro.distributions.HalfCauchy(0.3, 0.05).sample((100,)).tolist(), bins=100, binrange=(0,20)) 

In [12]:
one = run.fit(
    x=m, 
    k_list=k_list, 
    lr=0.005, 
    n_steps=10, 
    cluster=cluster, 
    beta_fixed=cosmic.loc[["SBS1","SBS5"]], 
    # hyperparameters={"alpha_sigma":.05, "alpha_p_sigma":1., "alpha_p_conc0":0.6, "alpha_p_conc1":0.6, "alpha_rate":1., "pi_conc0":0.5},
    enforce_sparsity = True, 
    reg_weight = 0., 
    store_parameters=False, 
    verbose = False, 
    save_runs_seed=True,
    seed=[92],
    new_hier = False, 
    nonparametric = False,
    save_all_fits=True
    )

obj = one[0] 
# 5 = n_samples, 1,2 + 2 = n_sigs, 2 = n_groups 

Running model with 4 signatures, 4 groups and 92 seed

N parameters 1312
Running model with 5 signatures, 4 groups and 92 seed

N parameters 1562


In [15]:
obj.all_fits["K_5.G_4"]

In [ ]:
# obj.inf_groups = obj.groups
# obj.groups = obj.init_params["init_clusters"].tolist() 

In [72]:
print({i:obj.groups.count(i) for i in set(obj.groups)})
print({i:g_rare.count(i) for i in set(g_rare)})
print({i:g.count(i) for i in set(g)} ) 

print("\n", obj.k_denovo)


{0: 26, 1: 55, 2: 38, 3: 31}
{1: 55, 2: 45, 3: 42, 4: 8}
{1: 63, 2: 45, 3: 42}

 4


In [73]:
print(sklearn.metrics.normalized_mutual_info_score(obj.groups, g_rare))
print(sklearn.metrics.adjusted_rand_score(obj.groups, g_rare), "\n")
print(sklearn.metrics.normalized_mutual_info_score(obj.groups, g))
print(sklearn.metrics.adjusted_rand_score(obj.groups, g)) 


0.7105893115261042
0.6619089758400103 

0.7509727025956865
0.7421423676094429


In [54]:
obj.init_params["pi_param"] 

tensor([0.5733, 0.1333, 0.1667, 0.1267], dtype=torch.float64)

In [74]:
obj.pi 

[0.9911840040963293,
 0.0021281413622185703,
 0.003026402029660946,
 0.00366145251179123]

In [56]:
obj.params["pi_conc0"] 

tensor([0.1578, 0.6601, 0.7742])

In [ ]:
idxs = [i for i, v in enumerate(g_rare) if v == 4]
print([v for i, v in enumerate(obj.groups) if i in idxs])


In [ ]:
sns.scatterplot(x=range(len(obj.losses)), y=obj.losses)

In [ ]:
try: sns.scatterplot(x=range(len(obj.gradient_norms["alpha_prior_param"])), y=obj.gradient_norms["alpha_prior_param"]) 
except: print()

In [ ]:
try: sns.scatterplot(x=range(len(obj.gradient_norms["alpha_noise_param"])), y=obj.gradient_norms["alpha_noise_param"])
except: print()

In [ ]:
try: sns.scatterplot(x=range(len(obj.gradient_norms["alpha"])), y=obj.gradient_norms["alpha"]) 
except: print()

In [ ]:
try: sns.scatterplot(x=range(len(obj.gradient_norms["beta_denovo"])), y=obj.gradient_norms["beta_denovo"])
except: print()

In [ ]:
try: sns.scatterplot(x=range(len(obj.gradient_norms["pi_param"])), y=obj.gradient_norms["pi_param"]) 
except: print()

In [ ]:
pd.DataFrame(np.array(obj.init_params["alpha_prior_param"]), columns=obj.alpha.columns).plot.bar(stacked=True, legend=False) 

In [ ]:
try: pd.DataFrame(np.array(obj.alpha_prior), columns=obj.alpha.columns).plot.bar(stacked=True, legend=False) 
except: print()

In [ ]:
try: pd.DataFrame(np.array(obj.alpha_prior_unn), columns=obj.alpha.columns).plot.bar(stacked=True)
except: print() 

In [ ]:
# print(obj.alpha_prior) 
# print(obj.init_params["alpha_t_param"]) 
# print(pyro.param("alpha_prior_param")) 

In [ ]:
try:
    for gid in set(obj.groups):
        tmp = [i for i, v in enumerate(obj.groups) if v == gid]
        pd.DataFrame(np.array(obj.alpha_noise), columns=obj.alpha.columns).iloc[tmp].plot.bar(stacked=True) 
    # pd.DataFrame(np.array(obj.alpha_noise), columns=obj.alpha.columns).iloc[idxs].plot.bar(stacked=True, legend=False)
except:
    print() 

In [ ]:
try:
    for gid in set(obj.groups):
        tmp = [i for i, v in enumerate(obj.groups) if v == gid]
        # tmp = [i for i, v in enumerate(obj.groups) if (v == gid and i in idxs)]
        if len(tmp) == 0: continue
        pd.DataFrame(np.array(obj.alpha), columns=obj.alpha.columns, index=obj.alpha.index).iloc[tmp].plot.bar(stacked=True)
except:
    obj.alpha.plot.bar(stacked=True, legend=False) 

In [ ]:
print(obj.alpha_noise.min()) 
print(obj.alpha_noise.max()) 

In [ ]:
obj.pi

In [ ]:
a = torch.tensor([-.3,.3,.3,.5])
print(torch.min(a))
print(a / torch.sum(a)) 
print(a - torch.min(a)) 
print( (a - torch.min(a)) / torch.sum(a - torch.min(a)) ) 

In [ ]:
[0, .6, .6, .8]

In [ ]:
isinstance(obj.pi, torch.Tensor)

In [ ]:
obj.post_probs

In [ ]:
obj.groups 

In [ ]:
obj.alpha

In [ ]:
obj.k_denovo

In [ ]:
obj.k_fixed

In [ ]:
obj.alpha_prior_unn

In [ ]:
obj.alpha_noise

In [ ]:
obj.alpha_unn + obj.alpha_noise 

In [ ]:
obj.alpha 

In [ ]:
from pyro import distributions as distr
import seaborn as sns
import numpy as np

# sns.displot(distr.HalfNormal(obj.hyperparameters["alpha_var"]).sample_n(1000).tolist())
# sns.displot(distr.Exponential(obj.hyperparameters["exp_rate"]).sample_n(1000).tolist()) 
vars = distr.Normal(0, 0.005).sample_n(50).tolist()
centrs = [distr.Normal(0.5, np.abs(v_i)).sample().tolist() for v_i in vars]

pl = sns.scatterplot(x=vars, y=centrs)
pl.axhline(0.5-0.05)
pl.axhline(0.5+0.05)

# sns.displot(vars)
# sns.displot(distr.Normal(0, 0.01).sample_n(1000).tolist()) 

In [ ]:
sns.displot(distr.Normal(0, 0.005).sample_n(1000).tolist()) 

In [ ]:
x2 = one.epsilon.astype(int)

In [ ]:
two = run.fit(
    x=x2, 
    k_list=[1,2,3,4,5], 
    lr=0.05, 
    n_steps=500, 
    enumer=False, 
    cluster=None, 
    groups=None, 
    beta_fixed=None, 
    compile_model = False, 
    CUDA = False, 
    enforce_sparsity = False, 
    regularizer = "cosine", 
    reg_weight = 1, 
    reg_bic = False, 
    store_parameters=False, 
    verbose=False, 
    stage = "two"
    )

In [ ]:
two.alpha

In [ ]:
import torch
import numpy as np

m1 = torch.tensor(m.values).float()
alpha1 = torch.tensor(one.alpha.values).float()
a1 = torch.matmul(torch.diag(torch.sum(m1, axis=1)), alpha1)

m2 = torch.tensor(x2.values).float()
alpha2 = torch.tensor(two.alpha.values).float()
a2 = torch.matmul(torch.diag(torch.sum(m2, axis=1)), alpha2)

b1 = torch.sum(a1, 1).unsqueeze(-1)
b2 = torch.sum(a2, 1).unsqueeze(-1)
b3 = b1 + b2

alpha1 = a1 / b3
alpha2 = a2 / b3

In [ ]:
sample_names = list(one.alpha.index)
fixed_names = list(one.alpha.columns)
denovo_names = list(two.alpha.columns)
alpha1 = pd.DataFrame(np.array(alpha1), index=sample_names, columns=fixed_names)
alpha2 = pd.DataFrame(np.array(alpha2), index=sample_names, columns=denovo_names)
alpha = pd.concat([alpha1, alpha2], axis=1)

In [ ]:
alpha

In [ ]:
torch.sum(torch.tensor(alpha.values).float(), 1).unsqueeze(-1)